In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
import openpyxl
import requests
import qrcode

def extract_hyperlink_urls(file_path, sheet_names, column_index):
    workbook = openpyxl.load_workbook(file_path)

    all_urls = {}

    for sheet_name in sheet_names:
        sheet = workbook[sheet_name]
        urls = []

        for row in sheet.iter_rows(min_row=2, min_col=column_index):
            cell = row[0]

            # Check if the cell contains a hyperlink
            if cell.hyperlink:
                url = cell.hyperlink.target
                urls.append(url)

        all_urls[sheet_name] = urls

    return all_urls

def download_logos(urls, folder):
    for sheet_name, urls_list in urls.items():
        for idx, url in enumerate(urls_list):
            response = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
            if response.status_code == 200:
                logo_path = os.path.join(folder, f"logo_{sheet_name[0]}_{idx + 1}.png")
                with open(logo_path, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded logo from {url} and saved as {logo_path}")
            else:
                print(f"Failed to download logo from {url}")

def generate_qrcode(urls, folder):
    for sheet_name, urls_list in urls.items():
        for idx, url in enumerate(urls_list):
            qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=10, border=4)
            qr.add_data(url)
            qr.make(fit=True)
            img = qr.make_image(fill_color="black", back_color="white")
            qr_path = os.path.join(folder, f"qrcode_{sheet_name[0]}_{idx + 1}.png")
            img.save(qr_path)
            print(f"Generated QR code for {url} and saved as {qr_path}")

# Manual input 
file_path = '' 
sheet_names = []

# Download logos from hyperlink in column D and save in "qr_codes_logos" folder
logo_column_index = 4  # Column D (index 4) contains hyperlinks for logos
logo_folder = 'logos'
if not os.path.exists(logo_folder):
    os.makedirs(logo_folder)

logo_hyperlink_urls = extract_hyperlink_urls(file_path, sheet_names, logo_column_index)
download_logos(logo_hyperlink_urls, logo_folder)

# Generate QR codes for hyperlinks in column E and save in "qr_codes_qr" folder
qr_column_index = 5  # Column E (index 5) contains hyperlinks for QR codes
qr_folder = 'qr_codes'
if not os.path.exists(qr_folder):
    os.makedirs(qr_folder)

qr_hyperlink_urls = extract_hyperlink_urls(file_path, sheet_names, qr_column_index)
generate_qrcode(qr_hyperlink_urls, qr_folder)
